In [ ]:
import os.path as path
from os import makedirs, chmod
import glob
import functools

import time
import datetime
import tqdm

import aopy
import ecog_is2s

import numpy as np
import torch
import torch.optim as optim
from torchvision.transforms import Compose

import matplotlib.pyplot as plt

import sys

sys.path.append("C:\\Users\\mickey\\aoLab\\code\\py4sid")
import estimation

In [ ]:
# file list to dataset
data_path_root = 'C:\\Users\\mickey\\aoLab\\Data\\WirelessData\\Goose_Multiscale_M1'
data_path_day = path.join(data_path_root,'18032*')
# data_file_list = glob.glob(path.join(data_path_day,'0[0-9]*\\*ECOG*clfp_ds250_fl0u10.dat'))
data_file_list = glob.glob(path.join(data_path_day,'0[0-9]*\\*ECOG*clfp_ds250.dat'))
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print('mounting to device: {}'.format(device))
print(f'files found:\t{len(data_file_list)}')
print(f'files: {data_file_list}')
datafile_list = [aopy.data.DataFile(df) for df in data_file_list]

In [ ]:
src_t = 0.5
trg_t = 0.25
step_t = src_t+trg_t
in_mem = True
diff_transform = ecog_is2s.Util.add_signal_diff() # no need for the srate parameter, dx est. is z-scored as well
zscore_transform = ecog_is2s.Util.local_zscore()
# transform = lambda sample : diff_transform(zscore_transform(sample))
transform = zscore_transform
dfds_list = [aopy.data.DatafileDataset(df,src_t,trg_t,step_t,device=device,in_mem=in_mem) for df in datafile_list]
datafile_concatdataset = aopy.data.DatafileConcatDataset(dfds_list,transform=transform)

In [ ]:
red_dim = 10
lags = 10
for sample_idx, (src, trg) in enumerate(datafile_concatdataset):
    Ahat, Chat = estimation.estimate_parameters_moments(src,lags,red_dim)
    Chat_pinv = np.linalg.pinv(Chat)
    breakpoint()
    enc_hat = src.numpy @ Chat_pinv

In [ ]:
(src.numpy() @ Chat_pinv.T).shape

In [ ]:
ex_idx = 1000
src, trg = datafile_concatdataset.__getitem__(ex_idx)
Ahat = np.linalg.pinv(src[:-1,:]) @ src[1:,:].numpy()
trg_hat = np.zeros(trg.shape)
trg_hat[0,:] = src[-1,:] @ Ahat
for idx in range(1,trg.shape[0]):
    trg_hat[idx,:] = trg_hat[idx-1,:] @ Ahat
t_src = np.arange(src.shape[0])/datafile_concatdataset.srate
t_trg = np.arange(trg.shape[0])/datafile_concatdataset.srate + src_t
plot_ch_idx = -1
plt.plot(t_src,src[:,plot_ch_idx],label='src')
plt.plot(t_trg,trg[:,plot_ch_idx],label='trg')
plt.plot(t_trg,trg_hat[:,plot_ch_idx],label='trg_hat')
plt.xlabel('time (s)')
plt.ylabel('a.u.')
plt.title(f'sample {sample_idx}')
plt.legend(loc=0)

In [ ]:
data = datafile_list[0].read()

In [ ]:
lags = 10
red_dim = 30
Ahat, Chat = estimation.estimate_parameters_moments(data,lags,red_dim)

In [ ]:
a = np.random.randn(10,5)
b = np.cov(a)
b.shape